# Vraag 2

## De vraag
Do countries in similar regions, also show similar price differences? And if differences occur, can you find a potential explanation?

### Indeling van groepen landen:
Midden Oosten: Afghanistan, Iran, Iraq, Yemen, Syria, Lebanon, Pakistan, Palestine


## Doelen
Grafiek per product voor meerdere landen in een regio


In [49]:
# Importing bokeh and pandas (Although we dont need Pandas just yet)
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Title, DataRange1d, FactorRange, Range1d, LabelSet, Label, Span
from bokeh.models import Toggle, BoxAnnotation, CustomJS
from bokeh.io import show, output_file
from bokeh.layouts import layout

import pandas
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import math
import random

df = pandas.read_csv("../data/global_food_prices.csv")
df_conflict = pandas.read_csv("../data/Conflict_bbc_same_name.csv")


In [50]:
def interpolate(values, unknown=None):
    if (len(set(values)) <= 1) or len(values) == 0: return values
    #if list(set(values))[0]==unknown or len(values) == 0: return []
    for i in range(len(values)):
        if values[i] == unknown:
            ii = i
            x = unknown
            while x == unknown:
                ii -= 1
                if ii < 0: x = values[i]
                x = values[ii]
            ii = i
            y = unknown
            while y == unknown:
                ii -= 1
                if ii >= len(values): y = values[i]
                y = values[ii]
            values[i] = (x + y) / 2
    return values

In [75]:
def prices_per_month(product, events, countries, df):
    file_name = "../graphs/Vraag2/prices_per_month/region_" + product + "_price.html"
    output_file(file_name, title=product)
    df = df[df['product']==product]
    prices_list = []
    dates_list = []
    
    # verzameld alle prijzen van landen
    for country in countries:
        df_country = df[(df['country']==country)]
        nulpunt = list(df_country['Unnamed: 0'][:1])
        factors = []
        prices = list(df_country['price_in_dollars'])
        prices_list.append(prices)
        
        # produceert x-as waardes
        for i in range(len(df_country['year'])):
            month_per_year = (str((df_country['year'][i + nulpunt[0]])), str((df_country['month'][i + nulpunt[0]])))
            factors.append(month_per_year)
        dates_list.append(factors)
    
    start = 0
    end = 0
    color_list = []
    
    # geeft maximale y-waarde
    for price in prices_list:
        if price:
            pricefloat = [float(x) if x != "UNKNOWN" else 0 for x in price]
            if max(pricefloat) > end:
                end = max(pricefloat)
    
    end = end * 1.03
    m = []
    
    # bepaald langste jarenlijst
    for date in dates_list:
        if len(date) > len(m):
            m = date 
    begindate = int(m[0][0])
    f = figure(x_range=FactorRange(*m), width=750)
    f.y_range=DataRange1d(start=0, end=end)
    title = "Prices of " + product + " for the middle east in USD"
    f.title.text = title
    f.xaxis.axis_label="Time"
    f.yaxis.axis_label="Product price in USD"
    f
    
    # plot per land lijn
    for i in range(len(countries)):
        prices_list[i] = [float(x) if x != 'UNKNOWN' else x for x in prices_list[i]]
        r = lambda: random.randint(0,255)
        color = '#%02X%02X%02X' % (r(),r(),r())
        f.line(dates_list[i], interpolate(prices_list[i], 'UNKNOWN'), legend=countries[i], line_color = color)
    
    code = '''\
    if toggle.active
        object.background_fill_alpha = 1
        object.text_alpha = 1
        object.border_line_alpha = 1
    else
        object.background_fill_alpha = 0
        object.text_alpha = 0
        object.border_line_alpha = 0
    '''
    
    # plot events in grafiek
    labels = []
    callbacks = []
    toggle_names = []
    toggles = []
    for event in events:     
        month = int(event[0])
        year = int(event[1])
        country = str(event[2])
        date = str(year), str(month) 
        event_text = country + ": \n"
        loc = ((year - begindate) * 12) + month + 1.4*(year - begindate) - 0.5
        df_event = df_conflict[(df_conflict['Month']==month) & (df_conflict['Year']==year) & (df_conflict['Country']==country)]
        events_list = list(df_event['Event'])
        if events_list != []:
            
            
            span = Span(location=loc, dimension='height', line_color='black', line_dash='dashed', line_width=1)
            f.add_layout(span)
            
            for event_descr in events_list:
                event_text = event_text + event_descr + "\n"
            
            toggle_name = str(month) + '-' + str(year)
            toggle_names.append(toggle_name)
            label = Label(x=loc, y=random.uniform(end*0.1, end*0.9), text=event_text, render_mode='css',
                 border_line_color='black', border_line_alpha=0,
                 background_fill_color='white', background_fill_alpha=0, text_alpha = 0)
            
            labels.append(label)
            callbacks.append(CustomJS.from_coffeescript(code=code, args={}))
            f.add_layout(label)

    for i in range(len(labels)):
        callback1 = callbacks[i]
        
        toggle1 = Toggle(label=toggle_names[i], button_type="success", callback = callback1)
        callback1.args = {'toggle': toggle1, 'object': labels[i]}
        toggles.append(toggle1)
        
    f.toolbar.logo = None
    f.toolbar_location = None
    f.legend.click_policy="hide"
    show(layout([f], toggles))
    return True
    


In [78]:
middle_east = ["Afghanistan", "Iran  (Islamic Republic of)", "Iraq", "Yemen", "Syrian Arab Republic", "Lebanon", "Pakistan"]
products = ["Sugar", 'Wheat', "Wheat flour", "Rice", "Fuel (petrol-gasoline)"]


# Lijst van lijsten,
# [[month, year, country], [month, year , country]]
sugar_events = [["12", "2012", "Lebanon"], ["3", "2012", "Iran  (Islamic Republic of)"], ["4", "2013", "Iran  (Islamic Republic of)"],["7", "2013", "Iran  (Islamic Republic of)"],["5", "2016", "Iran  (Islamic Republic of)"],["3", "2012", "Yemen"],["7", "2013", "Yemen"],["4", "2014", "Yemen"],["2", "2015", "Yemen"],["1", "2013", "Syrian Arab Republic"],["1", "2015", "Syrian Arab Republic"],["9", "2013", "Syrian Arab Republic"],["1", "2016", "Syrian Arab Republic"],["9", "2016", "Syrian Arab Republic"]]
egg_events = [["9", "2012", "Lebanon"],["10", "2012", "Lebanon"],["3", "2013", "Lebanon"],["10", "2014", "Lebanon"],["1", "2015", "Lebanon"],["9", "2015", "Lebanon"],["3", "2016", "Lebanon"],["7", "2012", "Iran  (Islamic Republic of)"],["7", "2015", "Iran  (Islamic Republic of)"],["6", "2013", "Iran  (Islamic Republic of)"],["1", "2014", "Iran  (Islamic Republic of)"],["2", "2016", "Iran  (Islamic Republic of)"]]
wheat_events =[["8", "2009", "Yemen"],["5", "2012", "Afghanistan"],["1", "2015", "Afghanistan"],["7", "2010", "Afghanistan"], ["10", "2014", "Pakistan"]]
wheatflour_events = [["6", "2014", "Iraq"],["5", "2015", "Iraq"], ["7", "2016", "Iraq"],["9", "2009", "Yemen"],["4", "2010", "Yemen"],["2", "2011", "Yemen"],["11", "2011", "Yemen"],["2", "2015", "Yemen"],["11", "2015", "Yemen"],["9", "2013", "Syrian Arab Republic"],["2", "2015", "Syrian Arab Republic"],["11", "2015", "Syrian Arab Republic"],["8", "2016", "Syrian Arab Republic"]]
rice_events = [["11", "2011", "Syrian Arab Republic"],["1", "2013", "Syrian Arab Republic"],["8", "2013", "Syrian Arab Republic"],["9", "2013", "Syrian Arab Republic"],["1", "2015", "Syrian Arab Republic"],["5", "2015", "Syrian Arab Republic"],["10", "2016", "Syrian Arab Republic"],["1", "2015", "Iraq"],["10", "2015", "Iraq"],["5", "2016", "Iraq"]]
fuel_events = [["7", "2014", "Yemen"],["3", "2015", "Yemen"],["6", "2015", "Yemen"],["8", "2015", "Yemen"],["11", "2015", "Yemen"],["11", "2014", "Pakistan"],["1", "2016", "Pakistan"]]
test = [["1", "2013", "Yemen"],["1", "2014", "Yemen"],["1", "2015", "Yemen"],["1", "2016", "Yemen"],["1", "2017", "Yemen"]]


prices_per_month("Sugar", sugar_events, middle_east, df)
prices_per_month("Eggs", egg_events, middle_east, df)
prices_per_month("Wheat", wheat_events, middle_east, df)
prices_per_month("Wheat flour", wheatflour_events, middle_east, df)
prices_per_month("Rice", rice_events, middle_east, df)
prices_per_month("Fuel (petrol-gasoline)", fuel_events, middle_east, df)

True